# Tutorial SEGOMOE via web services

This notebook describes the usage of segomoe using web services exposed by WhatsOpt.

## Prerequisite

### wop (WhatsOpt command line interface)

You have to install the `wop` command:

```bash
pip install -U wop
```

You need to be logged in a WhatsOpt server either by using the <code>wop</code> command in a shell or the WhatsOpt API as follows:

In [1]:
from whatsopt.whatsopt_client import WhatsOpt
wop = WhatsOpt(url="https://ether.onera.fr/whatsopt")
ok = wop.login(echo=True)

# If you have any trouble with the previous command 
# then logout to ensure to start from a clean slate and retry
if not ok: 
    wop.logout() 

You have to set your API key.
You can get it in your profile page on WhatsOpt (https://ether.onera.fr/whatsopt).
Please, copy/paste your API key below then hit return (characters are hidden).


Your API key:  ································


Successfully logged in to remote WhatsOpt https://ether.onera.fr/whatsopt



To test if you are connected the following command should succeed

In [2]:
wop.check_versions()

WhatsOpt 1.22.3 requires wop >= 1.20.0
You are using wop 2.1.0b1


### SMT (Surrogate Modeling Toolbox)

SMT is not required per se, but is used in this notebook to get optimized LHS sampling method.

```bash
pip install smt
```

## Optimization without constraint

### Objective function

First we define the objective function we want to minimize

In [3]:
import numpy as np

def f_obj(x):
    """
    Function Six-Hump Camel Back
    2 global optimum y_opt =-1.0316 located at x_opt = (0.089842, -0.712656) or (-0.089842, 0.712656)
    https://www.sfu.ca/~ssurjano/camel6.html
    """
    x_ = np.atleast_2d(x)
    x1 = np.array(x_)[:, 0]
    x2 = np.array(x_)[:, 1]
    val = 4*x1**2-2.1*x1**4+1./3.*x1**6+x1*x2-4*x2**2+4*x2**4
    return np.atleast_2d(val).T

### Optimization

First we create an optimization context to use the SEGOMOE optimizer with the design space <code>xlimits</code>.

In [18]:
from whatsopt.optimization import Optimization

xlimits = [[-3, 3], [-2, 2]]
optim = Optimization(xlimits, options={"mod_obj__regr": "constant", "optimizer": "cobyla"})

We need to have an initial DOE (n_samples, nx) and the corresponding outputs y (n_samples, 1).

In [19]:
import numpy as np
# from smt.sampling_methods import LHS
# lhs = LHS(xlimits=np.array(xlimits), criterion='ese')
# xdoe = lhs(5)

xdoe = np.array([[-0.91502224,  1.89017506],
 [ 2.57253436,  0.83786997],
 [-2.32304511, -1.12821447],
 [ 1.65152355, -1.63067708],
 [ 0.0057155,  -0.31036381]])
ydoe = f_obj(xdoe)
print("Initial DOE")
print("xdoe={}".format(xdoe))
print("ydoe={}".format(ydoe))

Initial DOE
xdoe=[[-0.91502224  1.89017506]
 [ 2.57253436  0.83786997]
 [-2.32304511 -1.12821447]
 [ 1.65152355 -1.63067708]
 [ 0.0057155  -0.31036381]]
ydoe=[[37.11048572]
 [32.43194792]
 [16.82595609]
 [17.00496851]
 [-0.34983144]]


We initialize the optimizer with the inital DOE.

In [20]:
optim.tell_doe(xdoe, ydoe)

We trigger the optimization using the "ask and tell" interface.

In [21]:
# We loop using the iteration budget
n_iter = 1
for i in range(n_iter):
    x_suggested, status, x_best, y_best = optim.ask()
    print("{} x suggested = {} with status: {}".format(i, x_suggested, Optimization.STATUSES[status]))

    # compute objective function at the suggested point
    new_y = f_obj(x_suggested)
    print("new y = {}".format(new_y))

    optim.tell(x_suggested, new_y)
    if optim.is_solution_reached():
        print("Solution is reached")
        break

x_opt, y_opt = optim.get_result()
print(f"Found minimum y_opt = {y_opt} at x_opt = {x_opt}")

0 x suggested = [0.5105824516906532, -0.3895314720981267] with status: valid point
new y = [[0.19223091]]
Found minimum y_opt = [[-0.34983144]] at x_opt = [[ 0.0057155  -0.31036381]]


In [8]:
optim.get_result()

(array([[ 0.08984755, -0.71265286]]), array([[-1.03162845]]))

In [9]:
optim.get_history()

(array([[-0.91502224,  1.89017506],
        [ 2.57253436,  0.83786997],
        [-2.32304511, -1.12821447],
        [ 1.65152355, -1.63067708],
        [ 0.0057155 , -0.31036381],
        [ 0.51058108, -0.38952938],
        [-0.43397872, -0.34841607],
        [ 0.15895667, -0.22111638],
        [ 0.07747435, -0.34265178],
        [ 0.07463824, -0.41646938],
        [ 0.09964009, -0.76109891],
        [ 0.1106892 , -0.70313999],
        [ 0.03994732, -0.7144226 ],
        [ 0.09114955, -0.71228802],
        [ 0.0883047 , -0.71184589],
        [ 0.0899261 , -0.71232543],
        [ 0.08986774, -0.71255056],
        [ 0.08988833, -0.71265405],
        [ 0.08984755, -0.71265286],
        [ 0.08986795, -0.71265729]]),
 array([[37.11048572],
        [32.43194792],
        [16.82595609],
        [17.00496851],
        [-0.34983144],
        [ 0.19223289],
        [ 0.4056637 ],
        [-0.1214223 ],
        [-0.41711352],
        [-0.58231808],
        [-1.01119413],
        [-1.02901105],
  

For convenience, the previous optimization loop is available as the <code>run</code> method of the optimization object.

In [10]:
# to reset the initial DOE, otherwise optimization will go on from previous state
# optim.tell_doe(xdoe, ydoe)

# run the optimization loop again
# optim.run(f_obj, 15)

## Optimization with constraints

### Objective and constraints functions

We define objective and constraints function and we build a grouped function which allows to evaluate all in one go.

In [11]:
import numpy as np

# Objective
def G24(point):
    """
    Function G24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def f_grouped(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T

In [12]:
f_grouped([1, 2])

array([[-3., -2.,  2.]])

### Optimization

First we create an optimization context to use the SEGOMOE optimizer with the design space <code>xlimits</code> and the constraints specifications <code>cstr_specs</code>.

In [13]:
from whatsopt.optimization import Optimization

xlimits = [[0, 3], [0, 4]]
cstr_specs = 2*[{"type": '<', "bound": 0.0}]
optim = Optimization(xlimits, cstr_specs)

Constraint type can be iether <code><</code>, <code>=</code> or <code>></code>. A tolerance may be specified using the <code>tol</code> key. Constraints defaults are :

In [14]:
print(Optimization.DEFAULT_CSTR)

{'type': '<', 'bound': 0.0, 'tol': 0.0001}


In [15]:
import numpy as np
from smt.sampling_methods import LHS
lhs = LHS(xlimits=np.array(xlimits), criterion='ese')

xdoe = lhs(5)
ydoe = f_grouped(xdoe)
print("Initial DOE")
print("xdoe={}".format(xdoe))
print("ydoe={}".format(ydoe))

Initial DOE
xdoe=[[0.15323798 2.99759722]
 [1.42089032 3.70531155]
 [2.11944613 2.16863473]
 [2.93258199 0.64944028]
 [0.89151314 0.97427108]]
ydoe=[[ -3.1508352    0.83742592 -20.24502151]
 [ -5.12620187   0.3511429    1.9383689 ]
 [ -4.28808086   0.04045512  -1.71804047]
 [ -3.58202226 -16.30962187   0.58153747]
 [ -1.86578422  -2.97892835   0.76497741]]


In [16]:
optim.tell_doe(xdoe, ydoe)

In [17]:
optim.run(f_grouped, n_iter=20, with_best=True)

0 x suggested = [1.7518169688627327, 3.9147891853484142] with status: invalid point
x_best=[[2.119446131883831, 2.1686347324782353]]
y_best=[[-4.288080864362066]]
new y = [[-5.66660615  1.53673711  0.39237204]]
1 x suggested = [3.0, 2.645117760650228] with status: invalid point
x_best=[[2.119446131883831, 2.1686347324782353]]
y_best=[[-4.288080864362066]]
new y = [[ -5.64511776 -17.35488224   2.64511776]]
2 x suggested = [2.5468845233455153, 4.0] with status: invalid point
x_best=[[2.119446131883831, 2.1686347324782353]]
y_best=[[-4.288080864362066]]
new y = [[-6.54688452 -1.88007188  2.03485965]]
3 x suggested = [3.0, 3.9999999999999996] with status: invalid point
x_best=[[2.119446131883831, 2.1686347324782353]]
y_best=[[-4.288080864362066]]
new y = [[ -7. -16.   4.]]
4 x suggested = [2.4004480009925775, 3.0232239329821438] with status: invalid point
x_best=[[2.119446131883831, 2.1686347324782353]]
y_best=[[-4.288080864362066]]
new y = [[-5.42367193 -0.82479689  0.20323322]]
5 x sugge

(array([[2.3295202 , 3.17849309]]), array([[-5.50801329]]))

Note: if the suggested x has already been told or very close a previous suggestion (see <code>numpy.allclose</code>), the solution is considered to be reached.

In [18]:
optim.get_status()

'solution reached'

In [19]:
optim.get_history()

(array([[0.15323798, 2.99759722],
        [1.42089032, 3.70531155],
        [2.11944613, 2.16863473],
        [2.93258199, 0.64944028],
        [0.89151314, 0.97427108],
        [1.75181697, 3.91478919],
        [3.        , 2.64511776],
        [2.54688452, 4.        ],
        [3.        , 4.        ],
        [2.400448  , 3.02322393],
        [2.35912271, 3.08038738],
        [2.32585172, 3.18952055],
        [2.32960263, 3.17753993],
        [2.32953287, 3.17843365],
        [2.3295202 , 3.17849309]]),
 array([[-3.15083520e+00,  8.37425918e-01, -2.02450215e+01],
        [-5.12620187e+00,  3.51142896e-01,  1.93836890e+00],
        [-4.28808086e+00,  4.04551236e-02, -1.71804047e+00],
        [-3.58202226e+00, -1.63096219e+01,  5.81537466e-01],
        [-1.86578422e+00, -2.97892835e+00,  7.64977412e-01],
        [-5.66660615e+00,  1.53673711e+00,  3.92372035e-01],
        [-5.64511776e+00, -1.73548822e+01,  2.64511776e+00],
        [-6.54688452e+00, -1.88007188e+00,  2.03485965e+00],
